<a href="https://colab.research.google.com/github/PhongPhan2k4/Parallel_Merge_Sort/blob/main/Parallel_Merge_Sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [194]:
import time
import numpy as np
import threading
import multiprocessing
import pandas as pd

## Merge Function

In [190]:
def merge(left_half, right_half):
    i = j = k = 0
    merged = []
    while i < len(left_half) and j < len(right_half):
        if left_half[i] < right_half[j]:
            merged.append(left_half[i])
            i += 1
        else:
            merged.append(right_half[j])
            j += 1

    while i < len(left_half):
        merged.append(left_half[i])
        i += 1

    while j < len(right_half):
        merged.append(right_half[j])
        j += 1

    return merged

## Merge Sort

In [191]:
def merge_sort(arr):
    length = len(arr)
    if length <= 1:
        return arr
    middle = length // 2
    left = merge_sort(arr[:middle])
    right = merge_sort(arr[middle:])
    return merge(left, right)

## Parallel Merge Sort

In [192]:
def parallel_merge_sort(arr):
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left_thread = threading.Thread(target=merge_sort, args=(left,))
    right_thread = threading.Thread(target=merge_sort, args=(right,))

    left_thread.start()
    right_thread.start()

    left_thread.join()
    right_thread.join()

    return merge(left, right)

## Excute

In [198]:
sizes = [10, 100, 1000, 10000, 100000, 1000000]
merge_sort_time = []
parallel_merge_sort_time = []
for size in sizes:
  arr = np.random.randint(size, size=size)
  # excute merge sort
  start = time.time()
  sorted = merge_sort(arr)
  merge_sort_time.append(time.time() - start)
  # excute parallel
  start = time.time()
  sorted = parallel_merge_sort(arr)
  parallel_merge_sort_time.append(time.time() - start)

In [199]:
dict = {'Size' : sizes, 'Merge Sort' : merge_sort_time, 'Parallel Merge Sort' : parallel_merge_sort_time}

In [200]:
df = pd.DataFrame(dict)
df

,Size,Merge Sort,Parallel Merge Sort
0,10,0.000063,0.002686
1,100,0.000440,0.002689
2,1000,0.006206,0.006561
3,10000,0.055216,0.080192
4,100000,0.660998,0.726876
5,1000000,9.775742,10.713621
